In [2]:
import cvxpy as cp
x,y = cp.Variable(), cp.Variable()
f = 40*x+30*y
g = [x>=0, y>=0, 2*x+y<=320, x+y<=240]
sol = cp.Problem(cp.Maximize(f), g)
print("Solution:", sol.solve())
print("x:", x.value)
print("y:", y.value)

Solution: 8000.000000000001
x: 79.99999999999996
y: 160.00000000000006


In [1]:
import pandas as pd

# import data
data = pd.read_csv('port5.txt')

In [2]:
asset_info = pd.DataFrame(data.iloc[0:225, 0].str.split(" ",expand=True))
asset_info = asset_info.rename({0: "id", 1:"rate",2:"sd"},axis=1)

In [3]:
asset_corr = pd.DataFrame(data.iloc[225:, 0].str.split(" ",expand=True))
asset_corr = asset_corr.rename({0: "id", 1:"a1",2:"a2",3:"k"},axis=1)

In [4]:
import numpy as np

mu = np.array(asset_info["rate"])
sd = np.array(asset_info["sd"])

k = [] 

for ind in range(1,226):
    padding = [0]*(ind-1)
    vals = list(asset_corr.loc[asset_corr["a1"] == str(ind)]["k"])
    vals = [float(c) for c in vals]
    k.append(padding + vals)
iu = np.triu_indices(225,1)
il = (iu[1],iu[0])
k = np.array(k)
k[il] = k[iu]

In [5]:
C = [[None]*225 for _ in range(225)] 

for r_ind in range(225):
    for c_ind in range(225):
        if r_ind <= c_ind:
            cur_k = asset_corr.loc[(asset_corr["a1"] == str(r_ind+1)) & (asset_corr["a2"] == str(c_ind+1))]["k"]
            C[r_ind][c_ind] = float(cur_k) * float(sd[r_ind]) * float(sd[c_ind])
        else:
            C[r_ind][c_ind] = C[c_ind][r_ind]

In [10]:
C2 = [[None]*225 for _ in range(225)] 

for r_ind in range(225):
    for c_ind in range(225):
        if r_ind <= c_ind:
            cur_k = k[r_ind][c_ind]
            C2[r_ind][c_ind] = float(cur_k) * float(sd[r_ind]) * float(sd[c_ind])
        else:
            C2[r_ind][c_ind] = C2[c_ind][r_ind]

In [11]:
C2

[[0.0014359552359999998,
  0.0007551617654240099,
  0.000976816551819708,
  0.000940255888246902,
  0.0007126593513748559,
  0.0008880601214706398,
  0.001148553334723178,
  0.00073058127683988,
  0.000490695479915578,
  0.0012416539066685522,
  0.0005720649733086398,
  0.000836613572770176,
  0.00091011518090866,
  0.0010608445440661,
  0.00105003713017224,
  0.0010713715901064,
  0.00111257693853887,
  0.0007513671103219839,
  0.0007132176041155199,
  0.0009898853642842317,
  0.001023865326219376,
  0.000976668641761056,
  0.0010635683748659041,
  0.0008222953125925199,
  0.0010490066865121918,
  0.0008193680012440959,
  0.001084028246433648,
  0.00062920249954274,
  0.0010612206110188081,
  0.0008631396767558439,
  0.0009568773208481999,
  0.000993764884173972,
  0.001074479629687964,
  0.000892394124034624,
  0.0009009421681267918,
  0.000785928689339266,
  0.00055271754286336,
  0.0007429573640206398,
  0.0006650690143436719,
  0.00042256627407378996,
  0.00082148284098036,
  0.00

In [9]:
C

[[0.0014359552359999998,
  0.0007551617654240099,
  0.000976816551819708,
  0.000940255888246902,
  0.0007126593513748559,
  0.0008880601214706398,
  0.001148553334723178,
  0.00073058127683988,
  0.000490695479915578,
  0.0012416539066685522,
  0.0005720649733086398,
  0.000836613572770176,
  0.00091011518090866,
  0.0010608445440661,
  0.00105003713017224,
  0.0010713715901064,
  0.00111257693853887,
  0.0007513671103219839,
  0.0007132176041155199,
  0.0009898853642842317,
  0.001023865326219376,
  0.000976668641761056,
  0.0010635683748659041,
  0.0008222953125925199,
  0.0010490066865121918,
  0.0008193680012440959,
  0.001084028246433648,
  0.00062920249954274,
  0.0010612206110188081,
  0.0008631396767558439,
  0.0009568773208481999,
  0.000993764884173972,
  0.001074479629687964,
  0.000892394124034624,
  0.0009009421681267918,
  0.000785928689339266,
  0.00055271754286336,
  0.0007429573640206398,
  0.0006650690143436719,
  0.00042256627407378996,
  0.00082148284098036,
  0.00

# QP Optimization

In [17]:
import cvxpy as cp

x = cp.Variable(225)
f = 0.5*cp.quad_form(x,C2)
g = [0.002<=mu@x, 0<=x, sum(x) == 1]
sol = cp.Problem(cp.Minimize(f), g)
print("Solution:", sol.solve())
print("x:", x.value)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
0.002 <= var1 * var3733 , because the following subexpressions are not:
|--  var1 * var3733